In [212]:
# importing all libraries.
import pandas as pd
import statsmodels.api as sm
import math
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from statsmodels.formula.api import ols
from sklearn import preprocessing
from scipy import stats

%matplotlib inline
plt.style.use('seaborn')

from sklearn.model_selection import train_test_split,KFold,cross_validate
from sklearn.linear_model import LinearRegression

In [213]:
# dropping extra index column.
df_clean_1 = pd.read_csv('df_clean_1.csv')
df_clean_1 = df_clean_1.drop(['Unnamed: 0'], axis=1)

In [214]:
# creating numpy arrays for the latitudes and longitudes of each house that is by the water.
water_lat = np.array(df_clean_1.loc[(df_clean_1['waterfront_bool'] == 1)]['lat'])
water_long = np.array(df_clean_1.loc[(df_clean_1['waterfront_bool'] == 1)]['long'])

# creating numpy arrays for latitudes and longitudes of each house.
house_lat = np.array(df_clean_1['lat'])
house_long = np.array(df_clean_1['long'])
house_id = np.array(df_clean_1['id'])

In [215]:
# defining function to find the closest waterfront house (on a given axis) for each house, returning the distance
# on the axis as well as the ID of the closest waterfront house.
def find_pairs(water_array, house_array, house_id):
    pairs_found = []
    cross_array = np.absolute(water_array[np.newaxis, :] - house_array[:, np.newaxis])
    for i in range(cross_array.shape[0]):
        row_min = cross_array[i].min()
        house_min_array = np.where(cross_array[i] == cross_array[i].min())
        house_min_index = np.array(house_min_array)[0]
        house_min_col = house_min_index.item(0)
        nearest_wh_on_axis = house_id[house_min_col]
        pairs_found.append((row_min, nearest_wh_on_axis))
    return pairs_found

In [216]:
find_pairs(water_lat,house_lat,house_id)

[(0.00019999999999953388, 6197800045),
 (0.004499999999993065, 3793500160),
 (0.0024999999999977263, 8035350320),
 (0.0015000000000000568, 9558200045),
 (0.000499999999995282, 1328310370),
 (0.00460000000000349, 7231300125),
 (0.017899999999997362, 8079040320),
 (0.005400000000001626, 7129300520),
 (0.0002000000000066393, 9545230140),
 (0.0035999999999987153, 7214720075),
 (0.0034999999999953957, 1202000200),
 (0.0033999999999991815, 1531000030),
 (0.00030000000000285354, 1853000400),
 (0.0034999999999953957, 4232902595),
 (0.00010000000000331966, 9547205180),
 (0.00019999999999953388, 3626039271),
 (0.0011999999999972033, 2767602356),
 (0.0037999999999982492, 2391600320),
 (0.018699999999995498, 8079040320),
 (0.00010000000000331966, 7895500070),
 (0.00030000000000285354, 2008000270),
 (0.0, 722079104),
 (0.0009999999999976694, 8079040320),
 (0.00019999999999953388, 1243100136),
 (0.0008999999999872443, 3454800060),
 (0.0227999999999966, 8079040320),
 (0.0007000000000019213, 929730005

In [217]:
find_pairs(water_long,house_long,house_id)

[(0.0, 9558200045),
 (0.003999999999990678, 9545240070),
 (0.0020000000000095497, 7766200013),
 (0.0, 3444100400),
 (0.013999999999995794, 5547700270),
 (0.054000000000002046, 5547700270),
 (0.0010000000000047748, 5104520400),
 (0.007999999999995566, 9545240070),
 (0.01099999999999568, 1321400060),
 (0.027999999999991587, 5547700270),
 (0.03300000000000125, 1202000200),
 (0.0049999999999954525, 3303700376),
 (0.001999999999995339, 7766200013),
 (0.013999999999995794, 5547700270),
 (0.0010000000000047748, 3444100400),
 (0.0, 6414100192),
 (0.0969999999999942, 5547700270),
 (0.0049999999999954525, 1321400060),
 (0.0, 1175000570),
 (0.016999999999995907, 9545240070),
 (0.007000000000005002, 1321400060),
 (0.0020000000000095497, 7766200013),
 (0.02400000000000091, 2799800710),
 (0.027000000000001023, 2799800710),
 (0.021000000000000796, 2799800710),
 (0.001999999999995339, 3717000160),
 (0.0040000000000048885, 2768000400),
 (0.009000000000000341, 9545240070),
 (0.016999999999995907, 330370

In [218]:
# creating a DF for the results of the function.
df_lat_pairs = pd.DataFrame(find_pairs(water_lat,house_lat,house_id), columns=['y_distance','nearest_w_house_on_y'])
df_long_pairs = pd.DataFrame(find_pairs(water_long,house_long,house_id), columns=['x_distance','nearest_w_house_on_x'])

# creating a DF for the house ID information.
df_id_info = pd.DataFrame(df_clean_1['id'].copy())
df_id_info['id_y'] = df_clean_1['lat']
df_id_info['id_x'] = df_clean_1['long']

In [219]:
# merging the new DFs.
df_pairs = pd.concat([df_id_info,df_lat_pairs,df_long_pairs], axis=1)
df_pairs.head(3)

,id,id_y,id_x,y_distance,nearest_w_house_on_y,x_distance,nearest_w_house_on_x
0,7129300520,47.5112,-122.257,0.0002,6197800045,0.000,9558200045
1,6414100192,47.7210,-122.319,0.0045,3793500160,0.004,9545240070
2,5631500400,47.7379,-122.233,0.0025,8035350320,0.002,7766200013


In [220]:
# creating new column to identify the axis on which the nearest waterfront house overall is.
df_pairs['nearest_axis'] = np.where(df_pairs['y_distance'] > df_pairs['x_distance'],'x','y')
df_pairs.head(3)

,id,id_y,id_x,y_distance,nearest_w_house_on_y,x_distance,nearest_w_house_on_x,nearest_axis
0,7129300520,47.5112,-122.257,0.0002,6197800045,0.000,9558200045,x
1,6414100192,47.7210,-122.319,0.0045,3793500160,0.004,9545240070,x
2,5631500400,47.7379,-122.233,0.0025,8035350320,0.002,7766200013,x


In [221]:
# creating a new column for the id of the nearest overall waterfront house.
df_pairs['nearest_w_overall'] = np.where(df_pairs['nearest_axis']=='x',df_pairs['nearest_w_house_on_x'],df_pairs['nearest_w_house_on_y'])

df_pairs.head(3)

,id,id_y,id_x,y_distance,nearest_w_house_on_y,x_distance,nearest_w_house_on_x,nearest_axis,nearest_w_overall
0,7129300520,47.5112,-122.257,0.0002,6197800045,0.000,9558200045,x,9558200045
1,6414100192,47.7210,-122.319,0.0045,3793500160,0.004,9545240070,x,9545240070
2,5631500400,47.7379,-122.233,0.0025,8035350320,0.002,7766200013,x,7766200013


In [222]:
# creating new extract DFs to lookup the coordinates for the neareast overall waterfront house.
df_ext = pd.DataFrame(df_pairs['nearest_w_overall'])
df_ext2 = df_ext.rename(columns={"nearest_w_overall": "id"})

df_id = pd.DataFrame(df_pairs[['id','id_y','id_x']])

In [223]:
# joining the extract DFs to lookup the nearest overall waterfront house coordinates.
df_nearest_coords = df_ext2.join(df_id.set_index('id'), on='id')
df_nearest_coords_2 = df_nearest_coords.drop(['id'], axis=1)
df_nearest_coords_3 = df_nearest_coords_2.rename(columns={"id_y": "nearest_y_coord","id_x": "nearest_x_coord"})

# tagging it onto the main DF.
df_pairs_2 = pd.concat([df_pairs,df_nearest_coords_3], axis=1)

In [224]:
# Filling in the minimal distances on each axes.
df_pairs_2['min_x'] = np.where(df_pairs_2['nearest_axis']=='x',
                               df_pairs_2['x_distance'],
                               abs(df_pairs_2['id_x']-df_pairs_2['nearest_x_coord']))
df_pairs_2['min_y'] = np.where(df_pairs_2['nearest_axis']=='y',
                               df_pairs_2['y_distance'],
                               abs(df_pairs_2['id_y']-df_pairs_2['nearest_y_coord']))
df_pairs_2.head(3)

,id,id_y,id_x,y_distance,nearest_w_house_on_y,x_distance,nearest_w_house_on_x,nearest_axis,nearest_w_overall,nearest_y_coord,nearest_x_coord,min_x,min_y
0,7129300520,47.5112,-122.257,0.0002,6197800045,0.000,9558200045,x,9558200045,47.4366,-122.335,0.000,0.0746
1,6414100192,47.7210,-122.319,0.0045,3793500160,0.004,9545240070,x,9545240070,47.5343,-122.054,0.004,0.1867
2,5631500400,47.7379,-122.233,0.0025,8035350320,0.002,7766200013,x,7766200013,47.4450,-122.347,0.002,0.2929


In [225]:
# converting distance of latitude from degrees to kilometers using the formula:
# 1 degree of latitude = 110.574km.
df_pairs_2['min_y_km'] = df_pairs_2['min_y']*110.574

# converting latitude from degrees to radians.
df_pairs_2['min_y_rad'] = df_pairs_2['min_y']*math.pi/180

# converting distance of longitude from degrees to kilometres using the formula:
# 1 degree of longitude = 111.320 * latitude in radians.
df_pairs_2['min_x_km'] = df_pairs_2['min_x']*111.320*df_pairs_2['min_y_rad']


In [228]:
# finding overall distance using pythagoras' theorem.
df_pairs_2['distance_to_water'] = ((df_pairs_2['min_y_km']**2)+(df_pairs_2['min_y_rad']**2))**0.5
df_pairs_2.head(3)

,id,id_y,id_x,y_distance,nearest_w_house_on_y,x_distance,nearest_w_house_on_x,nearest_axis,nearest_w_overall,nearest_y_coord,nearest_x_coord,min_x,min_y,min_y_km,min_y_rad,min_x_km,distance_to_water
0,7129300520,47.5112,-122.257,0.0002,6197800045,0.000,9558200045,x,9558200045,47.4366,-122.335,0.000,0.0746,8.248820,0.001302,0.000000,8.248821
1,6414100192,47.7210,-122.319,0.0045,3793500160,0.004,9545240070,x,9545240070,47.5343,-122.054,0.004,0.1867,20.644166,0.003259,0.001451,20.644166
2,5631500400,47.7379,-122.233,0.0025,8035350320,0.002,7766200013,x,7766200013,47.4450,-122.347,0.002,0.2929,32.387125,0.005112,0.001138,32.387125


In [230]:
# extracting small DF for use in regression model.
df_distance = df_pairs_2[['id', 'distance_to_water']]
df_distance.head(5)

,id,distance_to_water
0,7129300520,8.248821
1,6414100192,20.644166
2,5631500400,32.387125
3,2487200875,18.764408
4,1954400510,0.055287


In [232]:
df_distance.to_csv('df_distance.csv')